In [1]:
# 🔌 Cargar la extensión de Kedro en Jupyter
%load_ext kedro.ipython
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

[09/15/25 18:24:41] INFO     Using                                                                  ]8;id=212353;file://C:\Users\AlexandreKz\Documents\GitHub\Ml_Bank\venv\Lib\site-packages\kedro\framework\project\__init__.py\__init__.py]8;;\:]8;id=259448;file://C:\Users\AlexandreKz\Documents\GitHub\Ml_Bank\venv\Lib\site-packages\kedro\framework\project\__init__.py#270\270]8;;\
                             'C:\Users\AlexandreKz\Documents\GitHub\Ml_Bank\venv\Lib\site-packages\                
                             kedro\framework\project\rich_logging.yml' as logging configuration.                   

                    INFO     Registered line magic '%reload_kedro'                                   ]8;id=597825;file://C:\Users\AlexandreKz\Documents\GitHub\Ml_Bank\venv\Lib\site-packages\kedro\ipython\__init__.py\__init__.py]8;;\:]8;id=815183;file://C:\Users\AlexandreKz\Documents\GitHub\Ml_Bank\venv\Lib\site-packages\kedro\ipython\__init__.py#63\63]8;;\

                    INFO     Registered line magic '%load_node'                                      ]8;id=750168;file://C:\Users\AlexandreKz\Documents\GitHub\Ml_Bank\venv\Lib\site-packages\kedro\ipython\__init__.py\__init__.py]8;;\:]8;id=869147;file://C:\Users\AlexandreKz\Documents\GitHub\Ml_Bank\venv\Lib\site-packages\kedro\ipython\__init__.py#65\65]8;;\

                    INFO     Resolved project path as:                                              ]8;id=25098;file://C:\Users\AlexandreKz\Documents\GitHub\Ml_Bank\venv\Lib\site-packages\kedro\ipython\__init__.py\__init__.py]8;;\:]8;id=322910;file://C:\Users\AlexandreKz\Documents\GitHub\Ml_Bank\venv\Lib\site-packages\kedro\ipython\__init__.py#180\180]8;;\
                             C:\Users\AlexandreKz\Documents\GitHub\Ml_Bank\proyecto-ml-sebastiancar                
                             rera-kevinvivanco.                                                                    
                             To set a different path, run '%reload_kedro <project_root>'                           

[09/15/25 18:24:43] INFO     Kedro is sending anonymous usage data with the sole purpose of improving ]8;id=360089;file://C:\Users\AlexandreKz\Documents\GitHub\Ml_Bank\venv\Lib\site-packages\kedro_telemetry\plugin.py\plugin.py]8;;\:]8;id=723751;file://C:\Users\AlexandreKz\Documents\GitHub\Ml_Bank\venv\Lib\site-packages\kedro_telemetry\plugin.py#243\243]8;;\
                             the product. No personal data or IP addresses are stored on our side. To              
                             opt out, set the `KEDRO_DISABLE_TELEMETRY` or `DO_NOT_TRACK` environment              
                             variables, or create a `.telemetry` file in the current working                       
                             directory with the contents `consent: false`. To hide this message,                   
                             explicitly grant or deny consent. Read more at                                        
                             https://docs.kedro.org/en/stable/configuration/telemetry.html                         

[09/15/25 18:24:44] INFO     Kedro project Proyecto_ML_SebastianCarrera_KevinVivanco                ]8;id=433942;file://C:\Users\AlexandreKz\Documents\GitHub\Ml_Bank\venv\Lib\site-packages\kedro\ipython\__init__.py\__init__.py]8;;\:]8;id=210773;file://C:\Users\AlexandreKz\Documents\GitHub\Ml_Bank\venv\Lib\site-packages\kedro\ipython\__init__.py#146\146]8;;\

                    INFO     Defined global variable 'context', 'session', 'catalog' and            ]8;id=261908;file://C:\Users\AlexandreKz\Documents\GitHub\Ml_Bank\venv\Lib\site-packages\kedro\ipython\__init__.py\__init__.py]8;;\:]8;id=76408;file://C:\Users\AlexandreKz\Documents\GitHub\Ml_Bank\venv\Lib\site-packages\kedro\ipython\__init__.py#147\147]8;;\
                             'pipelines'                                                                           

                    INFO     Registered line magic 'run_viz'                                        ]8;id=669793;file://C:\Users\AlexandreKz\Documents\GitHub\Ml_Bank\venv\Lib\site-packages\kedro\ipython\__init__.py\__init__.py]8;;\:]8;id=307433;file://C:\Users\AlexandreKz\Documents\GitHub\Ml_Bank\venv\Lib\site-packages\kedro\ipython\__init__.py#153\153]8;;\

In [2]:

# Cargar datasets intermedios (ESTOS YA TIENEN VARIABLES QUE CONSIDERAMOS IRRELEVANTES " UNNAMED / VAR REDUNDATE)
df_cleaned = catalog.load("NIV_cleaned_dataset")
df_customer = catalog.load("NIV_customer_agg")
df_rfm = catalog.load("NIV_RFM")


print("Cleaned:", df_cleaned.shape)
print("Customer:", df_customer.shape)
print("RFM:", df_rfm.shape)

[09/15/25 18:24:49] INFO     Loading data from NIV_cleaned_dataset (CSVDataset)...             ]8;id=37627;file://C:\Users\AlexandreKz\Documents\GitHub\Ml_Bank\venv\Lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=579050;file://C:\Users\AlexandreKz\Documents\GitHub\Ml_Bank\venv\Lib\site-packages\kedro\io\data_catalog.py#1046\1046]8;;\

[09/15/25 18:24:51] INFO     Loading data from NIV_customer_agg (CSVDataset)...                ]8;id=970528;file://C:\Users\AlexandreKz\Documents\GitHub\Ml_Bank\venv\Lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=996761;file://C:\Users\AlexandreKz\Documents\GitHub\Ml_Bank\venv\Lib\site-packages\kedro\io\data_catalog.py#1046\1046]8;;\

[09/15/25 18:24:52] INFO     Loading data from NIV_RFM (CSVDataset)...                         ]8;id=37767;file://C:\Users\AlexandreKz\Documents\GitHub\Ml_Bank\venv\Lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=867934;file://C:\Users\AlexandreKz\Documents\GitHub\Ml_Bank\venv\Lib\site-packages\kedro\io\data_catalog.py#1046\1046]8;;\

Cleaned: (984247, 6)
Customer: (838370, 11)
RFM: (838370, 6)


ELIMINAR NULOS DE DATASETS "CLEANED" Y "CUSTOMER"
Motivo: Cantidad menor al 1% (11 filas)

In [3]:
#Eliminar nulos porque son despreciales (7 o 10)
print("Shape antes de eliminar nulos:", df_cleaned.shape)
print("Nulos restantes por columna:\n", df_cleaned.isnull().sum())
df_cleaned = df_cleaned.dropna()
print("Shape después de eliminar nulos:", df_cleaned.shape)
print("Nulos restantes por columna:\n", df_cleaned.isnull().sum())


print("Shape antes de eliminar nulos:", df_customer.shape)
print("Nulos restantes por columna:\n", df_customer.isnull().sum())
df_customer = df_customer.dropna()
print("Shape después de eliminar nulos:", df_customer.shape)
print("Nulos restantes por columna:\n", df_customer.isnull().sum())



Shape antes de eliminar nulos: (984247, 6)
Nulos restantes por columna:
 TransactionID              0
CustomerID                 0
CustLocation               7
TransactionDate            0
TransactionTime            0
TransactionAmount (INR)    0
dtype: int64
Shape después de eliminar nulos: (984240, 6)
Nulos restantes por columna:
 TransactionID              0
CustomerID                 0
CustLocation               0
TransactionDate            0
TransactionTime            0
TransactionAmount (INR)    0
dtype: int64
Shape antes de eliminar nulos: (838370, 11)
Nulos restantes por columna:
 CustomerID        0
txn_count         0
total_spent       0
avg_spent         0
max_spent         0
avg_balance       0
first_txn_date    0
last_txn_date     0
recency_days      0
tenure_days       0
txn_per_day       0
dtype: int64
Shape después de eliminar nulos: (838370, 11)
Nulos restantes por columna:
 CustomerID        0
txn_count         0
total_spent       0
avg_spent         0
max_spent      

AQUI VAMOS A TRANSFORMAR LOS DATOS :D PORQUE ALGUNOS NO ESTAN EN EL FORMATO QUE DEBERIA YUPIIII

In [4]:
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 984240 entries, 0 to 984246
Data columns (total 6 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   TransactionID            984240 non-null  object 
 1   CustomerID               984240 non-null  object 
 2   CustLocation             984240 non-null  object 
 3   TransactionDate          984240 non-null  object 
 4   TransactionTime          984240 non-null  object 
 5   TransactionAmount (INR)  984240 non-null  float64
dtypes: float64(1), object(5)
memory usage: 52.6+ MB
